<a href="https://colab.research.google.com/github/devdhruvper/LegalBrain-VectorSearch-1/blob/main/LegalBrain-VectorSearch-1/scripts/Pinnecone_DT_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Importing and getting the necessary stuff

In [ ]:
!pip install git+https://github.com/devdhruvper/splade.git

In [ ]:
!pip install sentence_transformers
!pip3 install pinecone-client
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoTokenizer
import pinecone

In [ ]:
!pip3 install pinecone-client

In [ ]:
import pinecone

In [ ]:
!pip install --upgrade pinecone-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Setting up with api keys

In [ ]:
pinecone.init(
    api_key="f112db94-1b02-44ec-b1d7-a4cf165fad28",  # app.pinecone.io
    environment="us-east1-gcp"  # next to api key in console
)

#Encoding detector code

In [ ]:
import chardet

with open("/content/Pd1.txt", "rb") as f:
  data = f.read()

encoding = chardet.detect(bytes(data)).get("encoding")
print(encoding)

utf-8


In [ ]:
import chardet

with open("/content/Pd2.txt", "rb") as f:
  data = f.read()

encoding = chardet.detect(bytes(data)).get("encoding")
print(encoding)

utf-8


###opening files and storing data in variables

In [ ]:
with open("/content/Pd1.txt", "r",encoding="utf-8") as f:
  d1 = f.read()
  f.close()

with open("/content/Pd2.txt", "r",encoding="utf-8") as f:
  d2 = f.read()
  f.close()

In [ ]:
print(len(d2))

184899


In [ ]:
print(len(d1))

884988


In [ ]:
pinecone.list_indexes()

['criminal-laws']

In [ ]:
index=pinecone.Index('criminal-laws')

###index stats

In [ ]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5038}},
 'total_vector_count': 5038}

In [ ]:
index.fetch([f"{5037}"])['vectors'][f"{5037}"]['metadata']['context']

'  \nExplanation. —”Arms ” means articles of any description designed or adapted as weapons for \noffence or defence and includes f ire arms, sharp edged weapons, lathis, dandas and sticks.’.  \n*                                    *                                    *                                     *                               * \n–––––––––  \n '

In [ ]:
d3[3013]

{'id': '5037',
 'context': '  \nExplanation. —”Arms ” means articles of any description designed or adapted as weapons for \noffence or defence and includes f ire arms, sharp edged weapons, lathis, dandas and sticks.’.  \n*                                    *                                    *                                     *                               * \n–––––––––  \n '}

In [ ]:
# L=[]
# for i in range(1472):
#   L.append(len(index.fetch([f"{i}"])['vectors'][f"{i}"]['metadata']['context'].split()))

In [ ]:
# print(min(L),max(L),sum(L)/1472)

In [ ]:
data_left = d2
print(data_left)

In [ ]:
limit = 384

def chunker(contexts: list):
    chunks = []
    all_contexts = ''.join(contexts).split('.')
    chunk = []
    for context in all_contexts:
        chunk.append(context)
        if len(chunk) >= 3 and len('.'.join(chunk)) > limit:
            # surpassed limit so add to chunks and reset
            chunks.append('.'.join(chunk).strip()+'.')
            # add some overlap between passages
            chunk = chunk[-2:]
    # if we finish and still have a chunk, add it
    if chunk is not None:
        chunks.append('.'.join(chunk))
    return chunks

In [ ]:
chunks=chunker(data_left)
print(chunks[440])

13 [An officer who, with respect to 14 *** any territory or place not forming part of 15 [India or] Her 
Majesty’s Dominions, is a Political Agent there for , as defined in section 3, 16 [clause (43)], of the General 
                                                      
1. Subs. by the A.O. 1950, for “British”. 
2. The Words “of Her Majesty, or” Rep . ibid . 
3. Subs. by the A.O.


In [ ]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# check device being run on
if device != 'cuda':
    print("==========\n"+
          "WARNING: You are not running on GPU so this may be slow.\n"+
          "If on Google Colab, go to top menu > Runtime > Change "+
          "runtime type > Hardware accelerator > 'GPU' and rerun "+
          "the notebook.\n==========")

dense_model = SentenceTransformer(
    'msmarco-bert-base-dot-v5',
    device=device
)
dense_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

**Printing first 10 chunks**

In [ ]:
d3=[]
chunks = chunker(data_left)
for i, context in enumerate(chunks):
    d3.append({
            'id': f"{i+5038}",
            #Adding 1472 to make the proper id
            #Check the actual ids and correct this 1472 accordingly
            'context': context
        })

d3[:10]

Splade model

In [ ]:
from splade.models.transformer_rep import Splade

sparse_model_id = 'naver/splade-cocondenser-ensembledistil'

sparse_model = Splade(sparse_model_id, agg='max')
sparse_model.to(device)  # move to GPU if possible
sparse_model.eval()

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(sparse_model_id)

tokens = tokenizer(d3[0]['context'], return_tensors='pt')


In [ ]:
with torch.no_grad():
    sparse_emb = sparse_model(
        d_kwargs=tokens.to(device)
    )['d_rep'].squeeze()
sparse_emb.shape

torch.Size([30522])

In [ ]:
indices = sparse_emb.nonzero().squeeze().cpu().tolist()
print(len(indices))

118


In [ ]:
values = sparse_emb[indices].cpu().tolist()
sparse = {'indices': indices, 'values': values}
sparse

In [ ]:
idx2token = {idx: token for token, idx in tokenizer.get_vocab().items()}

sparse_dict_tokens = {
    idx2token[idx]: round(weight, 2) for idx, weight in zip(indices, values)
}
# sort so we can see most relevant tokens first
sparse_dict_tokens = {
    k: v for k, v in sorted(
        sparse_dict_tokens.items(),
        key=lambda item: item[1],
        reverse=True
    )
}
sparse_dict_tokens

In [ ]:
import pinecone


def builder(records: list):
    ids = [x['id'] for x in records]
    contexts = [x['context'] for x in records]
    # create dense vecs
    dense_vecs = dense_model.encode(contexts).tolist()
    # create sparse vecs
    input_ids = tokenizer(
        contexts, return_tensors='pt',
        padding=True, truncation=True
    )
    with torch.no_grad():
        sparse_vecs = sparse_model(
            d_kwargs=input_ids.to(device)
        )['d_rep'].squeeze()
    # convert to upsert format
    upserts = []
    for _id, dense_vec, sparse_vec, context in zip(ids, dense_vecs, sparse_vecs, contexts):
        # extract columns where there are non-zero weights
        indices = sparse_vec.nonzero().squeeze().cpu().tolist()  # positions
        values = sparse_vec[indices].cpu().tolist()  # weights/scores
        # build sparse values dictionary
        sparse_values = {
            "indices": indices,
            "values": values
        }
        # build metadata struct
        metadata = {'context': context}
        # append all to upserts list as pinecone.Vector (or GRPCVector)
        upserts.append({
            'id': _id,
            'values': dense_vec,
            'sparse_values': sparse_values,
            'metadata': metadata
        })
    return upserts

In [ ]:
len(d3)/7

103.0

In [ ]:
from tqdm.auto import tqdm

batch_size = 7

for i in tqdm(range(0, len(d3), batch_size)):
    # extract batch of data
    i_end = min(i+batch_size, len(d3))
    batch = d3[i:i_end]
    # pass data to builder and upsert
    index.upsert(builder(d3[i:i+batch_size]))

  0%|          | 0/103 [00:00<?, ?it/s]

In [ ]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5759}},
 'total_vector_count': 5759}